In [1]:
import pandas as pd

import sys
sys.path.append(r"C:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\myLibraries")
from MARScore.utils import *
from MARScore.score import MARSCore

from custom_score.utils import cleanString
from sklearn.cluster import SpectralClustering, Birch
from hdbscan import HDBSCAN

import matplotlib.pyplot as plt
import numpy as np

c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are

In [2]:
dataset = pd.read_json(r'C:\Pro\Stages\A4 - DVRC\Work\Datasets\pubmed\test.json', lines=True)
dataset = dataset[["article_text", "abstract_text"]]
cleaner = lambda x: ". ".join(x).replace("<S>", "").strip()
format_dot = lambda x: x.replace(" .", ".")
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].replace(regex=r"\[[^\]]*\]", value="")
dataset.loc[:,"article_text"] = dataset["article_text"].replace(regex=r"\[[^\]]*\]", value="")
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(cleaner)
dataset.loc[:,"article_text"] = dataset["article_text"].map(cleaner)
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(cleanString)
dataset.loc[:,"article_text"] = dataset["article_text"].map(cleanString)
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(format_dot)
dataset.loc[:,"article_text"] = dataset["article_text"].map(format_dot)
dataset = dataset.rename(columns={"abstract_text": "summary",
                        "article_text": "text"})

In [3]:
subset = dataset.iloc[:2, :]

### Corpus to Sentences

In [ ]:
import spacy

In [ ]:
def corpusToSentences(corpus):
    corpus_sentences = []
    nlp = spacy.load("en_core_web_sm")
    for indiv in corpus:
        doc = nlp(indiv)
        corpus_sentences.append([sent.text for sent in doc.sents])   
    return corpus_sentences

In [ ]:
res = corpusToSentences(subset["text"].tolist())

In [ ]:
print(res)

### Token to sentences

In [ ]:
def cleanMarkers(embs, labels, mode="all", ignore=["."]):
    """
    Removes vectors associated with noisy words such as stop words, punctuation, and BERT separator tokens.

    :param1 embs (list): List of words embeddings.
    :param2 labels (list): List of text token associated with each embedding.

    :output1 new_embs (list): Cleansed list of words embeddings.
    :output2 new_labels (list): Cleansed list of tokens.
    :output3 -1 (int): Error output.
    """
    token_indexes = [i for i in range(len(labels)) if (labels[i] != "[PAD]" and labels[i] != "[CLS]" and labels[i] != "[SEP]") or labels[i] in ignore]
    new_embs = [embs[i] for i in range(len(embs)) if i in token_indexes]
    new_labels = [labels[i] for i in range(len(labels)) if i in token_indexes]

    if mode == "all":
        return new_embs, new_labels
    elif mode == "emb":
        return new_embs
    elif mode == "lab":
        return new_labels
    else:
        return -1

In [ ]:
o, l = tokenizeCorpus(subset["text"][0])
v = vectorizeCorpus(o)

In [ ]:
v, l = cleanMarkers(v, l)

In [ ]:
print(" ".join(l))

In [ ]:
sentenced_tokens = corpusToSentences([" ".join(l)])
print(sentenced_tokens)

### Sentence Masks

In [ ]:
mask = [i for i, sent in enumerate(sentenced_tokens[0]) for _ in sent.split(" ")]

In [ ]:
mask

### Sentences clusters visualisation

In [4]:
ms = MARSCore(subset["text"].to_list(),
              subset["summary"].to_list(),
              precision_level="s",
              ratio=5,
              n_allowed_elements=7,
              extraction_method="concat_l4",
              model=BertModel.from_pretrained('aitslab/biobert_huner_gene_v1'),
              tokenizer=BertTokenizer.from_pretrained('aitslab/biobert_huner_gene_v1'))
              #clusterizer=AffinityPropagation(random_state=0))
ms.compute()

In [5]:
def sentences_map(ms, index):
    sentences_mask = [i for i, sent in enumerate(ms.sentenced_labels[index]) for _ in sent.split(" ")]
    sentences_map = {0: []}
    for cluster_index, smask in zip(ms.clusters_labels[index], sentences_mask):
        if cluster_index in sentences_map.keys():
            sentences_map[cluster_index].append(smask)
        else:
            sentences_map[cluster_index] = [smask]
    return sentences_map

In [10]:
smap = sentences_map(ms, 0)

In [12]:
inv_smap = {}
for k,v in smap.items():
    for x in v:
        inv_smap.setdefault(x,[]).append(k)


In [14]:
print(inv_smap)

{55: [0, -1, -1, -1, -1, 4, 4, 9, 9, 50, 36, 78, 48, 82, 66, 32, 17, 17, 29, 75, 28, 28, 25, 25, 14, 14, 14, 13, 13, 1, 1, 16, 19, 19, 31], 56: [0, 0, 4, 4, 4, 49, 9, 9, 9, 50, 51, 59, 48, 56, 66, 33, 17, 17, 76, 75, 28, 28, 25, 25, 14, 13, 13, 1, 1, 16, 16, 19, 19, 31, 31], 62: [0, 4, 53, 52, 48, 17, 28, 23, 25, 14, 13, 1, 18, 142, 16, 31], 67: [0, 0, -1, -1, -1, 4, 4, 9, 9, 9, 71, 51, 51, 51, 122, 159, 38, 177, 91, 26, 26, 26, 131, 131, 131, 42, 42, 128, 77, 77, 80, 127, 61, 102, 160, 160, 160, 48, 48, 149, 149, 149, 149, 149, 37, 46, 11, 17, 17, 132, 132, 132, 132, 132, 76, 76, 75, 75, 28, 28, 23, 25, 25, 14, 14, 13, 13, 1, 1, 16, 16, 19, 19, 31, 31], 71: [0, -1, 4, 126, 24, 143, 9, 50, 36, 150, 48, 97, 37, 46, 11, 17, 23, 134, 151, 25, 14, 13, 1, 18, 16, 31, 135], 72: [0, 0, -1, 4, 4, 9, 9, 100, 51, 159, 38, 27, 160, 48, 48, 32, 32, 17, 17, 28, 23, 23, 65, 65, 65, 72, 72, 72, 25, 25, 14, 14, 13, 13, 1, 1, 18, 18, 16, 16, 31, 31], 73: [0, 4, 9, 71, 53, 52, 160, 160, 48, 37, 46, 17, 